# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** The data set used is for the most part discrete numbers and binary information.  This coincides with the output type of supervised classification - discrete (class labels).  Regression output type is a number.  I don't think that is beneficial for predicting if a student needs an intervention.

Secondly, with the data provided, we are trying to find a decision boundary of "to intervene or not."  A best fit line, again, is for numerical values: such as house cost, income.  (NOTE: I am taking age as discrete.  It technically could be taking as continuous for regression purposes, but I currently don't see the benefit.)

Thirdly, our evaluation is judged on accuracy.  I currently don't see how to produce a sum of the squared error for the data.

Therefore, the supervised learning problem should be classification.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data["sex"])

# TODO: Calculate number of features
n_features = len(student_data.keys())-1 # I am subtracting one to remove the target column as a feature

# TODO: Calculate passing students
n_passed_array = []
n_failed_array = []
for x in student_data["passed"]:
    if x == 'yes':
        n_passed_array.append(x)
    else:
        n_failed_array.append(x)
n_passed = len(n_passed_array)

# TODO: Calculate failing students
n_failed = len(n_failed_array)

# TODO: Calculate graduation rate
grad_rate = (n_passed / float(n_students))*100 # Percentage of those that passed vs total number of students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn import cross_validation

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, test_size=num_test, random_state=1)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

1) Decision Trees - "a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features."

Decision trees are good for classification so it suits my needs.  Secondly, we have broken nearly every label into basically binary decisions and Decision Trees seems very convenient for this problem.

Decision trees are used in real world for numerous applications.  One application for decision trees is in biomedical engineering.  Decision trees were used for identifying features to be used in implantable devices as seen in this reference: https://www.ncbi.nlm.nih.gov/pubmed/8258439

In the following link shows numerous applications in which decisions trees are used: http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html

Advantages:"Simple to understand and to interpret. Trees can be visualised.
The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree."

This problem does not have a lot of data points, so it is beneficial to have a learning algorithm that does not need a lot of points for training. Also, our features are somewhat binary, so the vizualation is pretty to easy. 

Disadvantages:
SKLEARN DOC: "Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning (not currently supported), setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.
Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.
Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble."

These are all valid things to worry about.  Therefore, I need to keep these in mind when finalizing my data. Specifically overfitting.

2) Ensemble Method (AdaBoost) - "The core principle of AdaBoost is to fit a sequence of weak learners (i.e., models that are only slightly better than random guessing, such as small decision trees) on repeatedly modified versions of the data. The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction."

My first reason for using this classifier is I really enjoyed the class and liked the part about how Ensemble Learning with boost give us a very good chance of not overfitting. 

My second reason is for using AdaBoost is I find this problem very similar to the spam issue we discussed in class.  What are some rules that work with a set of students and what rules work with another.  We do not have a lot of data points, but we have a lot of features.  So lets make rules on some features to prevent the curse of dimensionality and then combine those rules decide which kids need help.

Ensemble Learning has been used in the real world. "Ensemble learning is just a way of combining the output of multiple models in order to get a better result. This is a cheap way of improving your model, so i would bet that most real world applications use some sort of ensemble.

This technique became popular after the Netflix Challenge where the winning teams used ensembles of a lot of models to win the competition. This challenge aimed to build a new recommender system for Netflix to propose new movies for their users." -https://www.quora.com/Can-you-give-some-specific-examples-of-Ensemble-Learning

Here is a portion from that winning paper:
"Gradient Boosted Decision Trees (GBDT) are an additive regression model consisting of an ensemble of trees, fitted to current residuals in a forward step-wise manner. In the traditional boosting framework, the weak learners are generally shallow decision trees consisting of a few leaf nodes. GBDT ensembles are found to work well when there are hundreds of such decision trees. Standard references are [5, 6], and a known implementation is Treenet [16]."

If you are interested in the entire paper (its pretty neat), it can be seen here:
http://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf

Advantages:
Should not overfit, as discussed in class.

Disadvantages:
Time and computation expensive.  
Hard to implement in real time platform.
Complexity of the classification increases.

Niculescu-Mizil and Caruana demonstrated that boosting (in its most popular forms like AdaBoost) delivers poor probability estimates. This can be seen here: https://arxiv.org/ftp/arxiv/papers/1207/1207.1403.pd

"Cornerstone of ensemble-learning is problem of diversity. The ideal ensemble systems should be accurate as possible and make errors on different parts of the input space => classifiers should be diverse. Although this definition is logic and clear, there are number of different measures of diversity and their relationship with ensemble accuracy is not straightforward.

The methods of construction of the ensemble systems that attempt to achieve the highest diversity by intuitive way, i.e. by weakening the weights of similar classifiers such as AdaBoost, work significantly better than the basic methods such as simple majority vote approach. Surprisingly, the experiments proved that the measuring of diversity and trying to explicitly use it during the iterative process of construction of the ensemble system has not worked nearly as well."
https://www.researchgate.net/post/Any_advice_on_the_limitations_involved_in_an_ensemble_of_classifiers

The two referenced disadvantages are definitely worth keeping in mind. However, I tend to make sure I know the weakness of a learner and not assume it to be until I have results that look like the disadvantages are happening.  

3) Support Vector Machines - "SVMs are a set of supervised learning methods used for classification, regression, and outlier detection."

As stated above, I am looking at this problem as a classification problem so this learning method works.

I have found a few examples of SVM used in the real world.  One example: "Protein remote homology detection is a central problem in computational biology. Supervised learning algorithms based on support vector machines are currently one of the most effective methods for remote homology detection. The performance of these methods depends on how the protein sequences are modeled and on the method used to compute the kernel function between them." 

I thought this was interesting because I have never heard of a homology before. BTW, Homology is similarity in sequence of a protein or nucleic acid between organisms of the same or different species.

Mores examples of how SVMs are used in the real world can be found here: http://www.clopinet.com/SVM.applications.html

Advantages
SKLEARN DOC: "Effective in high dimensional spaces.
Still effective in cases where number of dimensions is greater than the number of samples.
Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels."

As stated in the documentation, this learning method is good for high dimensional spaces.  This seems appropriate since we have such a large number of features.

Disadvantages or as I like to call them, "Things to worry about":
CLASS NOTES: "Sometimes with a lot of data, SVMs (out of the box) might be slow and could overfit.
SVMs don't work well with lots and lots of noise, so when the classes are overlapping, I have to count independent evidence."

For us, I do not think we have a lot of noise nor is our particular data set really big.

SKLEARN DOC: "If the number of features is much greater than the number of samples, the method is likely to give poor performances." - Ref: http://scikit-learn.org/stable/modules/svm.html

As shown above, we have 395 samples with 30 features.  That is an order of magnitude difference so I assume we don't have to work about that.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [13]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A - Decision Trees used
from sklearn import tree
# from sklearn import model_B
from sklearn.ensemble import AdaBoostClassifier
# from skearln import model_C
from sklearn import svm

# TODO: Initialize the three models
clf_A = tree.DecisionTreeClassifier(random_state=0)
clf_B = AdaBoostClassifier(random_state=0)
clf_C = svm.SVC(random_state=0)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
# Per recommendations, I have added the 'for' loop.

for size in [100, 200, 300]:
    train_predict(clf_A, X_train[:size], y_train[:size], X_test, y_test)
    print "-------------------------------------------------------"

print "-------------------------------------------------------"
for size in [100, 200, 300]:
    train_predict(clf_B, X_train[:size], y_train[:size], X_test, y_test)
    print "-------------------------------------------------------"

print "-------------------------------------------------------"
for size in [100, 200, 300]:
    train_predict(clf_C, X_train[:size], y_train[:size], X_test, y_test)
    print "-------------------------------------------------------"


Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6441.
-------------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7077.
-------------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0030 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6720.
-------------------------------------------------------
-------------------------------------------------------
Training a AdaBoostClassifier using a training set

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - DecisionTreeClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0010           |        0.0000          |      1.0000      |    0.6441       |
| 200               |        0.0020           |        0.0010          |      1.0000      |    0.7077       |
| 300               |        0.0030           |        0.0010          |      1.0000      |    0.6720       |

** Classifer 2 - AdaBoostClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.1060           |        0.0030          |      0.9624      |    0.6949       |
| 200               |        0.0810           |        0.0060          |      0.8633      |    0.7647       |
| 300               |        0.0890           |        0.0040          |      0.8578      |    0.8116       |

** Classifer 3 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0010           |        0.0010          |      0.8591      |    0.8333       |
| 200               |        0.0030           |        0.0010          |      0.8581      |    0.8408       | 
| 300               |        0.0070           |        0.0010          |      0.8584      |    0.8462       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
I chose the SVM as my Best Model.  Based on the scores above, the choice should be either AdaBoostClassifier or the SVMClassifier.  It should be noted that the scores above are calculated on a specific random state of the cross-validation.  So the score for the AdaBoostClassifier might be higher than the score for the SVM Classifier for different random states, but the trends of each score set based on the training set size is similar.

I choose the SVM as my best model because it's F1 Score is pretty consistent for all training and testing sizes.  Based on these scores it does not look as if the data is overfitting either. SVM is also quicker. 

Number of students brings up a good point to discuss. As stated previously, there are only 395 students in this data set.  According to NCES.ed.gov, "In fall 2016, about 50.4 million students will attend public elementary and secondary schools."  SVM might not be the best classifier if the number of students increase.  But for this problem, I like it best.

These scores are also with the classifier in the "box of the box" default settings.  With a little adustments to our SVC learner, we can get a better score for our students.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** 
A support vector machine is a supervised learning model that involves trying to split data up into groups (or classes).  It does this by splitting the data with a line (2D) or plane (3D-know as a hyperplane).  This hyperplane is trying to find the location that provides the most space between different groups, known as the *maximum-margin* line.  This line is in the middle of the two groups and it works well because it allows for noise or mistakes in the data.  It should be noted that the larger the margin, the better chance the learning model will not overfit the data.  Please see figure below for what a hyperplane will look like when splitting up the data.  There is two data classes in the figure below: blue circles and red squares.


In [14]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url= "https://assets.ec.quoracdn.net/main-qimg-312b52d9d056499a3c8893e22f6fee5e?convert_to_webp=true", width=400, height=400)

It is called __support vector__ machine because the points closest to the hyperplane hold the plane in place between the different groups or classes (maximum margin).  In the figure above, the support vectors are the filled in symbols (2 red squares and 1 blue circle).  These support vectors define the maximum-margin line.

There is another trick that the SVM uses to separate data.  In 2D space, sometimes it looks like the data cannot be separated with a straight line or hyperplane, as seen in the next image.

In [15]:
Image(url= "https://assets.ec.quoracdn.net/main-qimg-fdf28b833fe56da2e047ea2524d12686?convert_to_webp=true", width=400, height=400)

This is when the SVM uses what is called the Kernel Trick.  The Kernel Trick looks at the data from another dimension (this case the third) and it can be seen that some data is clustered higher and some data is clustered together in the lower section.  This will allow for a hyperplane to be drawn between the data and separate it accordingly.  As seen in the final image.

In [16]:
Image(url= "https://assets.ec.quoracdn.net/main-qimg-f491532a3668e2597c65a8693ed043fd?convert_to_webp=true", width=400, height=400)

Please see following link for reference to images: https://www.quora.com/What-does-support-vector-machine-SVM-mean-in-laymans-terms

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [17]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.grid_search import GridSearchCV

# TODO: Create the parameters list you wish to tune
C_range = [1, 5, 50, 75, 150, 200, 250, 300, 600]
gamma_range = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
#kernel_range = ['rbf', 'linear','poly','sigmoid']
tol_range = [1e-3, 1e-4, 1e-5, 1e-6]
param_grid = dict(gamma=gamma_range, C=C_range, tol=tol_range)

# TODO: Initialize the classifier
clf = svm.SVC(random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
from sklearn.cross_validation import StratifiedShuffleSplit
cv = StratifiedShuffleSplit(y_train, 10, test_size=95, random_state=0)
grid_obj = GridSearchCV(clf, param_grid=param_grid, scoring=f1_scorer, cv=cv)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0050 seconds.
Tuned model has a training F1 score of 0.8188.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.8591.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
Tuned model has a testing F1 score of 0.8591. This is about 1% pecentage point higher than the default SVM.  It is also 4.75% pecentage points higher than the AdaBoost.

It is worth noting that I used the StratifiedShuffleSplit cross-validation generator.  This was done because we are dealing with an unbalanced dataset.  Our target row has 67% of students passing and 33% failing.  Therefore the StratisfiedShuffleSplit was used because it creates splits by preserving the same percentage for each target class as in the complete set. - See ref: http://scikit-learn.org/stable/modules/cross_validation.html

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.